In [2]:
!pip3 install ragas
!pip3 install ragchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.0 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.5/863.5 kB 21.1 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 55.7 MB/s eta 0:00:000m eta 0:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 60.7 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 43.5 MB/s eta 0:00:000m eta 0:00:01:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from ragchecker import RAGResults, RAGChecker
from ragchecker.metrics import all_metrics
import requests
import pandas as pd

/home/ubuntu/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/myenv/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:
# Define the custom function
def my_llm_api_func(prompts):
    response_list = []
    for prompt in prompts:
        # Replace with your custom API endpoint and authentication
        api_url = "http://128.214.253.165:5000/get"
        payload = {"query": prompt}

        response = requests.post(api_url, json=payload)
        response_data = response.json()
        api_response = response_data.get("response", "")
        response_list.append(api_response)
    return response_list

In [9]:
# Load RAGResults from JSON
with open("RAGChecker_evaluation.json") as fp:
    rag_results = RAGResults.from_json(fp.read())

In [10]:
# Initialize evaluator with your custom LLM API function
evaluator = RAGChecker(
    custom_llm_api_func=my_llm_api_func,
    batch_size_extractor=32,
    batch_size_checker=32
)

# Evaluate results
evaluator.evaluate(rag_results, all_metrics)

# Print the results
print(rag_results)

2024-12-30 09:37:08.815 | INFO     | ragchecker.evaluator:extract_claims:113 - Extracting claims for gt_answer of 82 RAG results.
100%|██████████████████████████████████████████████████████████████████████████████| 3/3 [21:08<00:00, 422.99s/it]
2024-12-30 09:58:17.784 | INFO     | ragchecker.evaluator:check_claims:173 - Checking response2answer for 82 RAG results.
100%|██████████████████████████████████████████████████████████████████████████████| 3/3 [21:58<00:00, 439.40s/it]
2024-12-30 10:20:15.998 | INFO     | ragchecker.evaluator:extract_claims:113 - Extracting claims for response of 82 RAG results.
100%|██████████████████████████████████████████████████████████████████████████████| 3/3 [22:55<00:00, 458.56s/it]
2024-12-30 10:43:11.697 | INFO     | ragchecker.evaluator:check_claims:173 - Checking answer2response for 82 RAG results.
100%|██████████████████████████████████████████████████████████████████████████████| 5/5 [38:12<00:00, 458.59s/it]
2024-12-30 11:21:24.659 | INFO     | 

RAGResults(
  82 RAG results,
  Metrics:
  {
    "overall_metrics": {
      "precision": 34.9,
      "recall": 45.4,
      "f1": 33.5
    },
    "retriever_metrics": {
      "claim_recall": 15.2,
      "context_precision": 29.3
    },
    "generator_metrics": {
      "context_utilization": 25.7,
      "noise_sensitivity_in_relevant": 8.5,
      "noise_sensitivity_in_irrelevant": 19.4,
      "hallucination": 36.1,
      "self_knowledge": 21.8,
      "faithfulness": 42.1
    }
  }
)
